In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import time
import pickle

In [2]:
# 웹 드라이버를 버전에 맞게 자동으로 다운 받고 옵션을 추가해 줌
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
driver.get("https://www.siksinhot.com/search?keywords=%EC%84%9C%EC%9A%B8")

In [11]:
search_list = [
    "강동구","강서구","강남구","도봉구","양천구","마포구","서대문구","은평구","서초구","구로구","중구",
    "동대문구","중랑구","광진구","송파구","강북구","관악구"
]

In [8]:
driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div").click()

In [9]:
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys("강북구")
driver.find_element(By.CSS_SELECTOR,"#searchInput").send_keys(Keys.RETURN)

In [12]:
all_pages = []
for search in search_list:
    input_box = driver.find_element(By.CSS_SELECTOR,"#header > header > div.searchBox > div")
    input_box.click()
    time.sleep(4)
    
    search_input = driver.find_element(By.CSS_SELECTOR,"#searchInput")
    search_input.send_keys(search)
    search_input.send_keys(Keys.RETURN)
    time.sleep(4)
    try:
        while True:
            more_box = driver.find_element(By.CSS_SELECTOR, "#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > a")

            more_box.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except:
        page_source = driver.page_source
        all_pages.append(page_source)
        pass 
    
    

In [13]:
len(all_pages)

17

In [20]:
for page in all_pages:
    soup = bs(page,"lxml")
    soup.select()

TypeError: select() missing 1 required positional argument: 'selector'

In [15]:
url = "https://www.siksinhot.com/search?keywords=%EC%84%9C%EB%8C%80%EB%AC%B8%EA%B5%AC"
r = requests.get(url)
soup = bs(r.text,"lxml")


In [17]:
main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")

In [51]:
page_list = []

for page in all_pages:
    soup = bs(page,"lxml")
    main = soup.select("#main_search > div > article > section > div.div_search_menu > div:nth-child(1) > ul > li")
    for i in main:
        dict_list = {}
        title = i.select_one("h2").text
        location = i.select_one(".cate>a:first-child").text
        division = i.select_one(".cate>a:last-child").text
        view = i.select_one(".counting").text
        score = i.select_one(".score").text
        book_mark = i.select_one(".counting>span:nth-of-type(2)").text
        url = i.select_one("figure > a").get('href')
        address = i.select_one("figure > a > img").get('alt').split(",")[-1] if i.select_one("figure > a > img") is not None else "None"
        
    
        dict_list["제목"] = title 
        dict_list["지역"] = location
        dict_list["분류"] = division
        dict_list["조회수"] = view
        dict_list["평점"] = score
        dict_list["북마크"] = book_mark
        dict_list["url"] = url
        dict_list["주소"] = address
        
        page_list.append(dict_list)

In [52]:
page_list

[{'제목': '동구밖오리구이',
  '지역': '천호/강동/둔촌',
  '분류': '오리고기',
  '조회수': '869018',
  '평점': '3.7',
  '북마크': '18',
  'url': 'https://www.siksinhot.com/P/342684',
  '주소': ' 서울특별시 강동구 진황도로 227 하선빌딩'},
 {'제목': '양철북(강동구청점)',
  '지역': '천호/강동/둔촌',
  '분류': '곱창/대창',
  '조회수': '154918',
  '평점': '3.3',
  '북마크': '8',
  'url': 'https://www.siksinhot.com/P/252209',
  '주소': '양철북(강동구청점) '},
 {'제목': '강동호치민',
  '지역': '천호/강동/둔촌',
  '분류': '인도네시아',
  '조회수': '46365',
  '평점': '2.8',
  '북마크': '5',
  'url': 'https://www.siksinhot.com/P/480260',
  '주소': ' 서울특별시 강동구 천호대로 1089 복합시설동 '},
 {'제목': '강동KD부대찌개',
  '지역': '천호/강동/둔촌',
  '분류': '부대찌개',
  '조회수': '52013',
  '평점': '2.8',
  '북마크': '3',
  'url': 'https://www.siksinhot.com/P/373435',
  '주소': ' 서울특별시 강동구 올림픽로62길 7'},
 {'제목': '떡갈비연구소 강동점',
  '지역': '천호/강동/둔촌',
  '분류': '한정식',
  '조회수': '1475',
  '평점': '평가중',
  '북마크': '',
  'url': 'https://www.siksinhot.com/P/1303788',
  '주소': ' 서울특별시 강동구 양재대로111길 54'},
 {'제목': '이차돌 강동구청점',
  '지역': '천호/강동/둔촌',
  '분류': '소구이/불고기',
  '조회수': '2344',
 

In [54]:
data = pd.DataFrame(page_list)

In [59]:
data.to_csv("식신.csv",encoding="utf-8",index=False)

In [4]:
data = pd.read_csv("식신.csv")

In [87]:
data[data["제목"].str.contains("화원")]

,제목,지역,분류,조회수,평점,북마크,url,주소
1077,화원,건대/자양/화양,중식,73054,2.8,4.0,https://www.siksinhot.com/P/364544,서울특별시 광진구 능동로16길 56-6


In [5]:
detail_pages = [] 
for links in data["url"][1078:]:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.set_window_size(1920, 1080) # 웹 브라우저 해상도 조절
    driver.get(links)
    time.sleep(3)
    
    try:
        while True:
            more_box2 = driver.find_element(By.CSS_SELECTOR, "#storeView > div.container > div.store_review > a")

            more_box2.click()
            time.sleep(2)  # 클릭 후 로딩을 위한 대기 시간 (필요에 따라 조절)

    except: 
        page_source2 = driver.page_source
        detail_pages.append(page_source2)
        driver.quit()
    

In [12]:
with open("식신디테일정보.pkl","wb") as file:
    pickle.dump(all_pages,file)

In [8]:
with open("식신디테일정보.pkl","rb") as read_file:
    all_pages = pickle.load(read_file)

In [10]:
pwd

'c:\\bigdata\\self_practice'

In [3]:
len(all_pages)

1078

In [84]:
all_pages[-1]

'<html><head>\n        <title>화원 - 서울 강북, 건대 자양 화양 | 맛집검색 식신</title>\n        <meta data-react-helmet="true" charset="utf-8"><meta data-react-helmet="true" http-equiv="Content-Type" content="text/html; charset=utf-8"><meta data-react-helmet="true" name="title" content="화원 - 서울 강북, 건대 자양 화양 | 맛집검색 식신"><meta data-react-helmet="true" name="description" content="화원입니다. 맛난 중국요리 드셔봐~입니다. 서울 강북, 건대 자양 화양에 위치하고 있습니다. 영업시간은 주중 10:00 ~ 22:00입니다. 대표메뉴는 향라유슬/짜장면/짬뽕, 족발찜입니다."><meta data-react-helmet="true" name="keywords" content="맛집, 서울 강북, 건대 자양 화양맛집, 맛집, 맛집, 향라유슬/짜장면/짬뽕맛집, 일/중/아시아맛집, 화원"><meta data-react-helmet="true" name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi"><meta data-react-helmet="true" name="robots" content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1 "><meta data-react-helmet="true" name="format-detection" content="telephone=no"><meta data-react-helmet

In [83]:
for k in bs(all_pages[-1],"lxml").select(".info_text"):
    print(k.text)

매장소개중국인이 직접 운영하는 곳으로 가격이 저렴하고 정통 중국음식을 먹을 수 있는 곳! 중국인 유학생들이 많이 찾는 본연의 중국의 맛을 선사하는 곳이다. 건대 부근에 위치해 있어서 찾기도 쉽고 가격도 저렴하다. 영업시간주중10:00 ~ 22:00OPEN메뉴향라유슬/짜장면/짬뽕6,000 원족발찜15,000 원탕고기튀김볶음10,000 원주소서울특별시 광진구 능동로16길 56-6지번서울특별시 광진구 화양동 513-1전화번호02-454-1888


In [17]:
for i in all_pages[1]:
    soup = bs(i,"lxml")
    for j in soup.select(".info_text"):
        print(j.select_one(".inline-div"))
    

C:\Users\hi\AppData\Local\Temp\ipykernel_49204\673518190.py:2: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(i,"lxml")


In [7]:
len(detail_pages)

950

In [9]:
len(all_pages)

1078

In [10]:
all_pages.extend(detail_pages)

In [11]:
len(all_pages)

2028

In [47]:
driver.find_element(By.CSS_SELECTOR,"#storeView > div.container > div.store_review > a").click()